In [1]:
import pandas as pd

# Turn CSV into a pandas DataFrame

connectivity_tab = pd.read_csv('./1-s2.0-S0092867416312429-mmc5.csv', sep=',')

# Show first 10 rows of table

connectivity_tab[0:10]



,Unnamed: 0,A09a a1l Basin-2,A09a a1r Basin-2,A09b a1l Basin-1,A09b a1r Basin-1,A09c a1l Basin-4,A09c a1r Basin-4,A09g a1l Basin-3,A09g a1r Basin-3,Drunken-1 a1l,...,lch5-3 a1l,lch5-3 a1r,lch5-5 a1l,lch5-5 a1r,v'ch a1l,v'ch a1r,vchA/B a1l,vchA/B a1l.1,vchA/B a1r,vchA/B a1r.1
0,A09a a1l Basin-2,0,0,5,0,0,0,0,0,1,...,2,0,0,0,2,0,0,1,0,0
1,A09a a1r Basin-2,0,0,0,12,0,1,0,0,0,...,0,0,0,1,0,13,0,0,0,0
2,A09b a1l Basin-1,0,0,0,0,1,0,0,0,3,...,1,0,0,0,11,0,5,1,0,0
3,A09b a1r Basin-1,0,0,0,0,0,1,0,0,0,...,0,5,0,0,0,11,0,0,4,6
4,A09c a1l Basin-4,1,0,12,0,0,0,1,0,0,...,0,0,0,0,0,0,8,9,0,0
5,A09c a1r Basin-4,0,0,0,9,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,3
6,A09g a1l Basin-3,0,0,11,0,1,0,0,0,0,...,3,0,0,0,0,0,10,11,0,0
7,A09g a1r Basin-3,0,0,0,9,0,3,0,0,0,...,0,0,0,0,0,3,0,0,9,11
8,Drunken-1 a1l,0,1,2,0,0,0,0,0,0,...,1,0,0,0,6,0,0,13,0,0
9,Drunken-1 a1r,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,9,0,0,4,8


In [2]:
# Make a list of column names to use as keys in lookup
columns = list(connectivity_tab.columns)
columns[0:5] # Return first five columns

['Unnamed: 0',
 ' A09a a1l Basin-2',
 ' A09a a1r Basin-2',
 ' A09b a1l Basin-1',
 ' A09b a1r Basin-1']

In [3]:
# Make a new list without the first (unnamed) column [0]:

lookup_keys = columns[1:]


# A dictionary (key: value pairs) mapping names from table to FBbt ids

lookup = { 'A09a a1l Basin-2': 'FBbt:00111229',
           'A09a a1r Basin-2': 'FBbt:00111229',
           'A09b a1l Basin-1': 'FBbt:00111227',
           'A09b a1r Basin-1': 'FBbt:00111227',
           'A09c a1l Basin-4': 'FBbt:00111230',
           'A09c a1r Basin-4': 'FBbt:00111230'}  # ... This should be completed for all columns

# Make a set (uniqued list) of all values in lookup

neurons = set(lookup.values())
neurons

{'FBbt:00111227', 'FBbt:00111229', 'FBbt:00111230'}

In [14]:
# Make a dictionary with key - column header & value = template specification (first row of table).
# Meks first two columns

template_seed = { 'ID': 'ID', 'CLASS_TYPE': 'CLASS_TYPE',  'RDF_Type' : 
'TYPE' }

# The rest of the columns have the neuron id as the column name + template spec in row 1:
data_columns = { n : "C 'synapsed to' some %" for n in neurons }

# Add data columns to template seed

template_seed.update(data_columns)

# Create dataFrame for template
# from_records takes a list of dicts - one for each row.  We only have one row.

template = pd.DataFrame.from_records([template_seed])  



In [17]:
# Iterate over input table rows
## Iterate over rows.

# Using a dict of dicts as an intermediate data structure.  Surely better to assign directly to DataFrame?
rows = {}
for i, r in connectivity_tab.iterrows():
    input_row_key = r[0].lstrip() # use the first column as key for row.  Make sure no trailing/leading whitespace
    # Skip to next iteration if row key not in lookup:
    if not input_row_key in lookup.keys(): continue
    output_row_key = lookup[input_row_key]
    row_dict = { 'ID' : output_row_key, 'CLASS_TYPE' : 'subclass', 'RDF_Type' : 'owl:class'}
    counter = 0
    # iterate over row (k = key/column header, v = value of cell)
    for k,v in r.items():
        key = k.lstrip()  # Turns out that headers have a leading space. This strips it.
        counter += 1
        # Skip the first column
        if counter == 1:
            continue
        # check we can lookup key
        if key in lookup.keys():
            # lookup fbbt ID
            fbbt = lookup[key]
            # default assumption = 0 synapses
            row_dict[fbbt] = ''
            # over-ride default if v > 0 for any key mapping to fbbt
            # v is a string, we need to turn it into an integer before checking it's > 0
            if int(v) > 0:
                row_dict[fbbt] = fbbt
    rows[output_row_key] = row_dict
    
out = template.append(list(rows.values()))
out

,CLASS_TYPE,FBbt:00111227,FBbt:00111229,FBbt:00111230,ID,RDF_Type
0,CLASS_TYPE,C 'synapsed to' some %,C 'synapsed to' some %,C 'synapsed to' some %,ID,TYPE
0,subclass,FBbt:00111227,,FBbt:00111230,FBbt:00111229,owl:class
1,subclass,,,FBbt:00111230,FBbt:00111227,owl:class
2,subclass,FBbt:00111227,,,FBbt:00111230,owl:class


In [16]:
out.to_csv("Jovanovic_template.csv", sep=",", index=False)